In [ ]:
!pip install -U requests urllib3
!pip install -qU langchain-huggingface
!pip install langchain_milvus
!pip install sentence-transformers
!pip install langchain_community
!pip install langchain_openai

In [1]:
from langchain_huggingface import HuggingFaceEmbeddings

model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

/usr/local/lib/python3.9/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
2024-08-21 15:36:50.199455: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-21 15:36:50.199506: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-21 15:36:50.201630: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-21 15:36:50.209546: I tensorflow/core/platform/cpu_feature_guard.cc:182] This T

In [2]:
from langchain_milvus import Milvus

URI = "./patients.db"

vector_store = Milvus(
    embedding_function=embeddings,
    connection_args={"uri": URI},
)

In [3]:
from langchain_core.documents import Document

vector_store_saved = Milvus.from_documents(
    [Document(page_content="foo!")],
    embeddings,
    collection_name="patient_records",
    connection_args={"uri": URI},
)

In [4]:
vector_store_loaded = Milvus(
    embeddings,
    connection_args={"uri": URI},
    collection_name="patient_records",
)

In [5]:
from uuid import uuid4
from langchain_core.documents import Document

patient_document_1 = Document(
    page_content="Patient ID: 3a511581-ef4d-4acb-963c-b357f1689b1b | Patient Bram Terlouw, age 45, has a history of hypertension and diabetes. Currently taking metformin and lisinopril.",
    metadata={"source": "medical_record"},
)

patient_document_2 = Document(
    page_content="Patient ID: d766efd8-d6ee-4fa9-a98d-46cf0904a11c | Patient Karin Bufles recent blood tests show an HbA1c level of 7.5%, indicating poorly controlled diabetes.",
    metadata={"source": "medical_record"},
)

patient_document_3 = Document(
    page_content="Patient ID: daba9ee8-6d49-4911-a9bd-fab179dc650b | During the last consultation, Patient John Doe reported experiencing frequent headaches and dizziness.",
    metadata={"source": "medical_record"},
)

patient_document_4 = Document(
    page_content="Patient ID: 6b97cee3-b2ed-4ae2-b831-04ee9e16fddb | Patient Bram Boshart, is a young boy with heart descease and a little left food.",
    metadata={"source": "medical_record"},
)


documents = [
    patient_document_4,
]
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

['14eb8347-ce4a-42bf-9360-b0a9b4701860']

In [ ]:
results = vector_store.similarity_search(
    "Patient ID: 3a511581-ef4d-4acb-963c-b357f1689b1b, Bram Terlouw",
    k=1,
    filter={"source": "medical_record"},
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

In [8]:
results = vector_store.similarity_search_with_score(
    "3a511581-ef4d-4acb-963c-b357f1689b1b Bram Terlouw", k=1, filter={"source": "medical_record"}
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=1.193668] Patient ID: 6b97cee3-b2ed-4ae2-b831-04ee9e16fddb | Patient Bram Boshart, is a young boy with heart descease and a little left food. [{'pk': '14eb8347-ce4a-42bf-9360-b0a9b4701860', 'source': 'medical_record'}]


In [10]:
milvus_retriever = vector_store.as_retriever(search_type="mmr", search_kwargs={"k": 1})
milvus_retriever.invoke("6b97cee3-b2ed-4ae2-b831-04ee9e16fddb Bram Boshart", k=1, filter={"source": "medical_record"})

[Document(metadata={'pk': '14eb8347-ce4a-42bf-9360-b0a9b4701860', 'source': 'medical_record'}, page_content='Patient ID: 6b97cee3-b2ed-4ae2-b831-04ee9e16fddb | Patient Bram Boshart, is a young boy with heart descease and a little left food.')]

In [ ]:
from langchain_community.llms import Ollama
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

mistral_llm = Ollama(
        base_url="http://llm.ic-shared-llm.svc.cluster.local:11434",
        model="mistral",
        top_p=0.92,
        temperature=0.01,
        num_predict=512,
        repeat_penalty=1.03,
        callbacks=[StreamingStdOutCallbackHandler()]
    )

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI


retriever = milvus_retriever
llm = mistral_llm

system_prompt = (
    "Use the given context to answer the question. "
    "If you don't know the answer, say you don't know. "
    "Use three sentence maximum and keep the answer concise. "
    "Context: {context}"
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, prompt)
chain = create_retrieval_chain(retriever, question_answer_chain)

# chain.invoke({"input": "Hello"})